In [604]:
import pandas as pd
import numpy as np
import os
path = os.getcwd()


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,BaggingRegressor, StackingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score

In [605]:
train = pd.read_csv(path + "\\TRAIN.csv", index_col = 'index')
test = pd.read_csv(path + "\\TEST.csv", index_col = 'index')
ss = pd.read_csv(path + "\\sample_submission.csv")

In [606]:
train['time_stamp'] = pd.to_datetime(train['time_stamp'], unit = 'ms')
test['time_stamp'] = pd.to_datetime(test['time_stamp'], unit = 'ms')

In [607]:
train["hour"] = pd.to_datetime(train.time_stamp, format="%Y-%m-%d").dt.hour
train["minute"] = pd.to_datetime(train.time_stamp, format="%Y-%m-%d").dt.minute
train["day"] = pd.to_datetime(train.time_stamp, format="%Y-%m-%d").dt.day

test["hour"] = pd.to_datetime(test.time_stamp, format="%Y-%m-%d").dt.hour
test["minute"] = pd.to_datetime(test.time_stamp, format="%Y-%m-%d").dt.minute
test["day"] = pd.to_datetime(test.time_stamp, format="%Y-%m-%d").dt.day

In [608]:
df = pd.concat([train, test], axis = 0).reset_index(drop=True)
df['time_stamp'] = pd.to_datetime(df['time_stamp'], unit = 'ms')

In [609]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
le = LabelEncoder()
sl = MinMaxScaler()

In [610]:
# import geopy
# from geopy.extra.rate_limiter import RateLimiter
# from geopy.geocoders import Nominatim
# import geopy.distance


# geolocator = Nominatim(user_agent="http")
# states = df['source'].unique()
# d = dict(zip(states, pd.Series(states).apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))))
# lat =pd.DataFrame(d, index=['pickup_latitude','pickup_longitude']).T
# Lat= lat.reset_index()
# Lat['source']=Lat['index']
# Lat.drop(columns=['index'],inplace=True)

# df = pd.merge(df, Lat, how="left") 

# states = df['destination'].unique()
# d = dict(zip(states, pd.Series(states).apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))))
# lat =pd.DataFrame(d, index=['dropoff_latitude','dropoff_longitude']).T
# Lat= lat.reset_index()
# Lat['destination']=Lat['index']
# Lat.drop(columns=['index'],inplace=True)

# df = pd.merge(df, Lat, how="left") 

In [611]:
# def prepare_distance_features(df):
#     # Distance is expected to have an impact on the fare
#     df['longitude_distance'] = abs(df['pickup_longitude'] - df['dropoff_longitude'])
#     df['latitude_distance'] = abs(df['pickup_latitude'] - df['dropoff_latitude'])

#     # Straight distance
#     df['distance_travelled'] = (df['longitude_distance'] * 2 + df['latitude_distance'] * 2) ** .5
#     df['distance_travelled_sin'] = np.sin((df['longitude_distance'] * 2 * df['latitude_distance'] * 2) ** .5)
#     df['distance_travelled_cos'] = np.cos((df['longitude_distance'] * 2 * df['latitude_distance'] * 2) ** .5)
# #     df['distance_travelled_sin_sqrd'] = np.sin((df['longitude_distance'] * 2 * df['latitude_distance'] * 2) * .5) * 2
# #     df['distance_travelled_cos_sqrd'] = np.cos((df['longitude_distance'] * 2 * df['latitude_distance'] * 2) * .5) * 2

#     return df

# df = prepare_distance_features(df)

In [612]:
dict_ = {'Theatre District': 1, 'Fenway': 2, 'Beacon Hill': 3,
        'North End': 4, 'Northeastern University': 5, 'Financial District': 6,
        'Boston University': 7, 'Haymarket Square': 8, 'West End': 9,
        'South Station': 10, 'North Station': 11, 'Back Bay': 12}

df['source'] = df['source'].map(dict_)
df['destination'] = df['destination'].map(dict_) 

In [613]:
label_cols = ['cab_provider', 'cab_type']

df[label_cols] = df[label_cols].apply(le.fit_transform)

In [614]:
train_proc, test_proc = df[:train.shape[0]], df[train.shape[0]:].reset_index(drop=True)

In [615]:
target = 'fare'
time = 'time_stamp'
time_feats = ['hour', 'year', 'minute', 'day','dayofweek', 'month', 'weekday']

features = [col for col in train_proc.columns if col not in [target, time]]

In [616]:
from sklearn.model_selection import train_test_split, StratifiedKFold

trn, val = train_test_split(train_proc, test_size = 0.2, random_state = 1999)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test_proc[features]

In [617]:
%%time
lgb = XGBRegressor(random_state=1999)

lgb.fit(X_trn, y_trn)

preds = lgb.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))
print(f'mean_squared_log_error is : {error}')

mean_squared_log_error is : 1.7010355358395521
Wall time: 1.19 s


In [618]:
%%time
lgb = LGBMRegressor(random_state=1999)

lgb.fit(X_trn, y_trn)

preds = lgb.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))
print(f'mean_squared_log_error is : {error}')

mean_squared_log_error is : 1.7725443389889546
Wall time: 318 ms


---
# Feature Engineering

In [619]:
def join(train_, test_):
    df = pd.concat([train_, test_], axis = 0).reset_index(drop=True)
    return df

def split(df):
    train_, test_ = df[:train.shape[0]], df[train.shape[0]:].reset_index(drop=True)
    return train_, test_

In [620]:
#df['source_+_dest'] = df.apply(lambda x: x['source'] + x['destination'], axis = 1)
# It worked, but not logical so try later 1.6922603905441371

In [621]:
# Cross Validation for Boosting
def cross_val(regressor, train, test, features, name):
    N_splits = 5
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    folds = StratifiedKFold(n_splits = N_splits, shuffle = True,random_state = 1999)
    stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    for index, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
        print(f'\n================================Fold{index + 1}===================================')
        
        #### Train Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        
        #### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
        
        #### Test Set
        X_test = test[features]
        
#         if name != 'cat':
#             #### Scaling Data ####
#             scaler = StandardScaler()
#             _ = scaler.fit(X_trn)
#             X_trn = scaler.transform(X_trn)
#             X_val = scaler.transform(X_val)
#             X_test = scaler.transform(X_test)
        
        ############ Fitting #############
        _ = regressor.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 50, verbose = 100)
        
        ############ Predicting #############
        val_preds = np.abs(regressor.predict(X_val))
        test_preds = np.abs(regressor.predict(X_test))
        
        error = np.sqrt(mean_squared_error(y_val, val_preds))
        print(f'\n Root Log Mean Squared Error for Validation set is : {error}')
        
        oofs[val_idx] = val_preds
        preds += test_preds / N_splits
        
    total_error = np.sqrt(mean_squared_error(target_col, oofs))
    print(f'\n\Root Log Mean Squared Error for oofs is {total_error}')
    
    return oofs, preds

In [622]:
df.head()

,time_stamp,cab_provider,source,destination,distance,surge_multiplier,cab_type,fare,hour,minute,day
0,2018-11-26 03:40:46.318,0,7,1,3.03,1.0,4,34.0,3,40,26
1,2018-11-26 03:40:46.319,1,10,1,1.30,1.0,0,18.5,3,40,26
2,2018-11-26 03:40:46.320,1,1,2,2.71,1.0,9,19.5,3,40,26
3,2018-11-26 03:40:46.320,0,5,3,2.43,1.0,5,10.5,3,40,26
4,2018-11-26 03:40:46.320,1,1,2,2.71,1.0,10,32.0,3,40,26


In [631]:
df = join(train_proc, test_proc)


df['Cab_Count_Per_Days_Hour'] = df.groupby(['day', 'hour'])['source'].transform('count')
df['hour_surge'] = df.groupby(['cab_type'])['distance'].transform('mean')

train_feat, test_feat = split(df)

In [632]:
# df = join(train_feat, test_feat)



# temp = df.groupby(['cab_provider']).agg({'distance':['mean','min', 'max'],
#                                      'fare':['mean','sum', 'min', 'max'],
#                                     })

# temp.columns = ['cab_provider_'.join(x) for x in temp.columns]
# df = pd.merge(df,temp,on=['cab_provider'],how='left')



# temp = df.groupby(['source', 'destination']).agg({'distance':['count','mean','min', 'max'],
#                                      'fare':['count','mean','sum', 'min', 'max'],
#                                     })

# temp.columns = ['source_destination_'.join(x) for x in temp.columns]
# df = pd.merge(df,temp,on=['source', 'destination'],how='left')




# temp = df.groupby(['cab_type']).agg({'distance':['mean','min', 'max'],
#                                      'fare':['mean','sum', 'min', 'max'],
#                                     })

# temp.columns = ['cab_type_'.join(x) for x in temp.columns]
# df = pd.merge(df,temp,on=['cab_type'],how='left')


# train_feat, test_feat = split(df)  


In [633]:
features = [col for col in train_feat.columns if col not in [target, time]]

In [634]:
%%time
xgb = XGBRegressor(random_state = 1999)
xgb_oofs, xgb_preds = cross_val(xgb, train_feat, test_feat, features, 'xgb')


================================Fold1===================================
[0]	validation_0-rmse:13.13262
[99]	validation_0-rmse:1.70683

 Root Log Mean Squared Error for Validation set is : 1.7068273599171762

================================Fold2===================================
[0]	validation_0-rmse:13.08994
[99]	validation_0-rmse:1.65883

 Root Log Mean Squared Error for Validation set is : 1.6588337884451416

================================Fold3===================================
[0]	validation_0-rmse:13.16666
[99]	validation_0-rmse:1.61752

 Root Log Mean Squared Error for Validation set is : 1.6173941940753123

================================Fold4===================================
[0]	validation_0-rmse:13.08444
[99]	validation_0-rmse:1.60287

 Root Log Mean Squared Error for Validation set is : 1.6026512711259184

================================Fold5===================================
[0]	validation_0-rmse:13.17432
[99]	validation_0-rmse:1.62781

 Root Log Mean Squared Erro

In [636]:
dict(zip(features, xgb.feature_importances_*100))

{'cab_provider': 1.2506357,
 'source': 0.2194538,
 'destination': 0.26532236,
 'distance': 5.1875668,
 'surge_multiplier': 8.329879,
 'cab_type': 67.314705,
 'hour': 0.09740815,
 'minute': 0.1005377,
 'day': 0.16820502,
 'Cab_Count_Per_Days_Hour': 0.09272708,
 'hour_surge': 16.973555}

# Note Downs:

### trn_tst_split :
        1. XGB: 1.70
        2. LGB: 1.77
### cross_val : 
        1. XGB: 1.64
        2. LGB: 1.71

### Features With Cross Val:
        1. 'Cab_Count_Per_Days_Hour' : 1.6391
        2. 'Cab_count_per_source_dest : 1.6479069412565543
        3. 